# Regresión logística
La regresión logística (_Logistic Regression_) es un típo de regresión cuyo objetivo es determinar la probabilidad de que una instancia pertezca a una clase $y$, dado un conjunto de variables independientes $x_i$ que la definen. En este contexto, las instancias están representadas como un vector de variables independientes $x=\mathbb{R}^{n}$ y una clase $y=\{0,1\}$. Es decir:

$$P(y|x)=h(x)$$

En este contexto, la función seleccionada para hacer esta estimación por excelencia es la sigmoide.

$$sigmoid(z)=\frac{1}{1+e^{-z}}$$

## Ejercicio
Grafíque la función sigmoid con $z$ en el rango $[-6, 6]$.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

#Ingrese su código aquí


Además de que la imagen de esta función está en $(0, 1)$, la derivada de esta función es:
$$\frac{sigmoid(z)}{dz}=sigmoid(z)(1-sigmoid(z))$$
Lo que facilitaba su implementación.

En este contexto, $z$ es una convinación lineal de las variables $x$.


## Función de error
Para calcular el error, se utiliza la entropía cruzada entre el valor esperado y el valor obtenido.
$$CE(y,\hat{y})=\frac{\sum(-y*log(\hat{y})-(1-y)*log(1-\hat{y}))}{N}$$
En este contexto, la entropía curzada se interpreta como la información promedio (en bits) necesaria para determinar el valor de $y$ dado que se conoce el valor de $\hat{y}$.

__Nota__: Por simplicidad, se interpreta el logaritmo como logaritmo natural, pero se puede usar logaritmo en cualquier base, ya que solo afecta en una constante.

__Ejercicio Opcional__: Calcule  $\frac{d(CE(y,\hat{y}))}{d\hat{y}}$, $\frac{d(CE(y,\hat{y}))}{dw}$ $\frac{d(CE(y,\hat{y}))}{db}$. Asuma que $w$ es un escalar y considere que $y$ es $0$ o $1$. 

## Ejemplo
Para el ejemplo de regresión logística se utilizará el conjunto de datos de cancer de pecho provisto. Este conjuntos de datos fue recolectado por investigadores de la Universisda de Wisconsin y provisto por la [UCI](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). Para acceder al conjunto de datos, no es necesario descargarlo y convertirlo al formato, ya que en encuentra provisto por el módulo de [_sklearn.datasets_](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer) de la librería sickit-learn, que es una librería de _machine learning_ que se utilizará durante el curso.

El dataset tiene 569 instancias, con 30 atributos cada una. Las instancias pueden ser clasificadas entre Malignos y Benignos. El dataset está ligeramente desbalanceado, lo que significa que existen más instancias de una clase que de la otra. En partícular, 37,25% de las instancias son Malignas y 62,75% son Benignas. La siguiente tabla resume el conjunto de datos:

| Propiedad | Valor |
| --- | --- |
| Clases | 2 |
| Ejemplos por clase | 212(M-0), 357(B-1) | 
| Total de instancias | 569 |
| Dimensionalidad | 30|

El siguiente código:
1. Levanta los datos divididos en `x` (atributos) e `y` (clase).
1. Divide los datos en entrenamiento y testing.
1. Escala los datos de entrenamiento a valores entre 0 y 1.
1. Aplica las correcciones de escalado al conjunto de testing.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix


x, y = load_breast_cancer(True)
x_train = x[:500,:]
y_train = y[:500]
x_test = x[500:,:]
y_test = y[500:]

maxs = np.max(x_train, axis=0)
mins = np.min(x_train, axis=0)
x_train = (x_train - mins) / (maxs - mins)
x_test = (x_test - mins) / (maxs - mins)

### Visualizando el problema
Muchas veces la visualización del problema ayuda a entender sus carácteristicas. Sin embargo, visualizar elementos en 30 dimensiones no es posible. Para esto existen diferentes técnicas que nos permiten reducir la dimensionalidad de los elementos manteniendo ciertas propiedades. Para visualizar los datos utilizaremos T-distributed Stochastic Neighbor Embedding (t-SNE) [1], que reduce la dimensiones manteniendo la distribución estadistica de las distancias entre los puntos de datos. 

El siguiente código aplica t-SNE a los datos de entrenamiento y los grafica utilizando estrellas azules para los benignos y rojas para los malignos. Es importante notar que los puntos obtenidos varian de ejecución en ejecución ya que t-SNE es sensible a su inicialización.

[1] van der Maaten, L.J.P.; Hinton, G.E. (Nov 2008). ["Visualizing High-Dimensional Data Using t-SNE"](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). Journal of Machine Learning Research. 9: 2579–2605.

In [ ]:
from sklearn.manifold import TSNE

ts_rep = TSNE().fit_transform(x_train)
for point, label in zip(ts_rep, y_train):
    rep = 'b*' if label == 1 else 'r*'
    plt.plot([point[0]], [point[1]], rep)
plt.show()

Una vez visualizados los datos, hacemos una regresión logística sobre los datos. Para analizar el funcionamiento de la regresíon logística utilizaremos una matrix de confusión. En esta matriz las filas representan las clases reales y las columnas las clases predichas. En cada celda se encuentran la cantidad de instancias con la clase de la fila y la predicción asociada de la columna.

| | Predición: 0  | Predición: 1 |
| --- | --- | --- |
| __Real: 0__ | Verdaderos Negativos | Falsos Positivos |
| __Real: 1__ | Falsos Negativos | Verdaderos Positivos |

Esta matriz permite visualizar fácilmente los valores de:
* __Verdaderos Negativos__: cantidad de instancias clasificadas como negativas que efectivamente eran negativas. En nuestro ejemplo Maligno es la clase negativa porque tiene asociada el cero. Generalmente referido como TN (_True Negative_).
* __Verdaderos Positivos__: cantidad de instancias clasificadas como positivas que efectivamente eran positivas. En nuestro ejemplo Benigno es la clase poasitiva porque tiene asociada el uno. Generalmente referido como TP (_True Positive).
* __Falsos Negativos__: cantidad de instancias clasificadas como negativas, pero en la realidad era positivas. También llamado Error de Tipo II en estadística. Generalmente referido como FN (_False Negative_).
* __Falsos Positivos__: cantidad de instancias clasificadas como positivas, pero en la realidad era negativos. También llamado Error de Tipo I en estadística. Generalmente referido como FP (_False Positive_).


__Nota__: Como la regresión logística retorna un valor probabilistico se seleccionó un umbral de 0.5 para discernir entre clasificaciones positivas y negativas.

In [ ]:
import tensorflow as tf

'''Esta función dibuja bonita la matríz de confunsión.
'''
def show_confusion_matrix(cm, labels):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Matriz de confusión')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicho')
    plt.ylabel('Verdadero')
    for i, row in zip(range(len(cm)), cm):
        for j, val in zip(range(len(row)), row):
            ax.text(i, j, str(val), va='center', ha='center').set_backgroundcolor('white')
    plt.show()

rng = np.random
ciclos = 1000
learning_rate = 0.1

# Placeholder de las entradas
X = tf.placeholder(tf.float32, [None, 30])
Y = tf.placeholder(tf.float32, [None])

W = tf.Variable(rng.randn(30).astype(np.float32), name="weight")
b = tf.Variable(rng.randn(), name="bias")

# Modelo logístico
logreg = tf.add(tf.reduce_sum(tf.matmul(X, tf.expand_dims(W, axis=1)), axis=1), b)
logreg = tf.div(1.0, tf.add(1.0, tf.exp(-logreg)))
# Error de entropía cruzada
cost = tf.reduce_mean(-Y * tf.log(logreg) - (1 - Y) * tf.log(1 - logreg))
# Gradient descent
# minimize() sabe que hay que modificar W y b porque están configuradas como trainable=True por defecto
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializa las variables
init = tf.global_variables_initializer()

# Comenzar una sessión
with tf.Session() as sess:

    # Inicializar
    sess.run(init)

    y_pred = sess.run(logreg, feed_dict={X: x_test, Y:y_test})
    print('Error: {}'.format(sess.run(cost, feed_dict={X: x_test, Y:y_test})))
    show_confusion_matrix(confusion_matrix(y_test, y_pred > 0.5), labels=['Maligno', 'Benigno'])
    errors = []
    errors_test = []
    print('Entrenando')
    for epoch in range(ciclos):
        sess.run(optimizer, feed_dict={X: x_train, Y: y_train})
        errors.append(sess.run(cost, feed_dict={X: x_train, Y:y_train}))
        errors_test.append(sess.run(cost, feed_dict={X: x_test, Y:y_test}))
    print('Error en entrenamiento')
    plt.plot(range(ciclos), errors, 'b-', range(ciclos), errors_test, 'r-')
    plt.show()
    y_pred = sess.run(logreg, feed_dict={X: x_test, Y:y_test})
    print('Error: {}'.format(sess.run(cost, feed_dict={X: x_test, Y:y_test})))
    show_confusion_matrix(confusion_matrix(y_test, y_pred > 0.5), labels=['Maligno', 'Benigno'])
    print('El w final es {}'.format(sess.run(W)))
    print('El b final es {}'.format(sess.run(b)))

## Métricas
La matriz de confusión es buena para visualizar las capacidades de un clasificador, pero dificulta comparar sus capacidades contra otros modelos. Por este motivo, se han desarrollado diversas métricas para evaluar cuan bien funciona un clasificador. Algunas de esta son:
* Accuracy: Porcentaje de predicciones correctas $acc=\frac{TP+TN}{TP+TN+FP+FN}$
* Precision: Porcentaje de predicciones positivas correctas $precision=\frac{TP}{TP+FP}$
* Recall: Porcentaje de las instancias pertenecientas a la clase positiva que fueron clasificadas correctamentes $recall=\frac{TP}{TP+FN}$
* F1-measure: Media harmónica entre la Presicion y el Recall. $F1=2\frac{precision \times recall}{precision+recall}$
* Matthews correlation coefficient: Es la correlación entre las predicciones y los valores reales. 1 indica predicción perfecta, -1 indica total desacuerdo entre lo predicho y lo real, y 0 indica que el clasificador no aporta ninguna información $MCC = \frac{ TP \times TN - FP \times FN } {\sqrt{ (TP + FP) ( TP + FN ) ( TN + FP ) ( TN + FN ) } }$

### Ejercicio
Implemente las métricas y apliquelas sobre los resultados del clasificador de ejemplo.
__Recuerde__: en el futuro podrá utilizar las implementaciones provistas por scikit-learn.

In [ ]:
#Implemente aquí su solucion 

## Problema de OCR de dígitos
Para este trabajos utilizaremos el conjunto de datos conocido como [MNIST](http://yann.lecun.com/exdb/mnist/). Este conjunto de datos ya se encuentra dividido entre entrenamiento y testing. El problema consiste en clasificar imagenes de dígistos escritos a mano al dígito correspondiente.

| Propiedad | Valor |
| --- | --- |
| Clases | 10 |
| Tamaño de las imagenes | 28 X 28 |
| Instancias de entrenemiento | 60.000 |
| Instancias de entrenemiento | 10.000 |
| Valor mínimo de cada pixel | 0 |
| Valor máximo de cada pixel | 255 |

A continuación se carga el dataset y se dibujan los primeros 100 ejemplos del conjunto de entrenamiento.

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('100 primeros elementos del conjunto de entrenaimento')
f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(x_train[i + j*10, :, :], cmap='gray')
plt.show()

### Trabajo práctico
Construya un clasificador que dado una imagen determine a que dígito corresponde. Una vez entrenado el clasificador, calcule valores de diversas métricas, como accuracy, matriz de confusión. Utilice las implementaciones de [scikit_learn](http://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics).

Consideraciones:
1. Recuerde cambiar la forma de las instancias de matrices de 28 X 28 a vectores de 784 elementos y escalar el valor.
1. Puede implementar un clasificador binario por clase, siendo la clase positiva si la instancia es ese dígito y negativa en otro caso. Por ejemplo, el clasificador del dígito 5 solo considerará positivos las imagenes de cincos y negativas todo el resto.
    * __Opcional__: Por eficiencia, considere agregar todas las regresiones logísticas en una sola operación. Puede definir $W$ como una matrix de 784 X 10 y $b$ como un vector de 10 elementos.
1. Recuerde que dado un arreglo numpy, los operadores `==`, `>`, `<` son aplicados elemento a elemento y que la cohersion `float(True)` es `1.0` y `float(False)` es `0.0`. 
1. En caso de obtener errores `nan`, `-inf` o `inf`, puede deberse a que $\lim_{x\to 0}log(x) =-\infty$. Considere limitar los valores de la regresión logitica entre $[\epsilon, 1-\epsilon]$ con $\epsilon\backsim0$, usualmente $\epsilon=1e-6$. En código, `logreg = tf.clip_by_value(logreg, epsilon, 1 - epsilon)`.
1. Pruebe diferentes valores para el _learning rate_ y la cantidad de _epochs_.

In [ ]:
#Ingrese su código aquí. Considere usar diversas celdas para definir funciones y ejecutar.